<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---

# 랜덤포레스트(Random Forests)

### 1) 캐글 대회를 이어서 진행합니다. EDA, 데이터 전처리 부분을 업데이트 하세요.
- EDA는 항상 완벽할 수 없지요 한 주간 계속 같은 데이터로 과제를 진행하므로 부족한 부분을 추가하거나 논하세요.
- (지금은 feature engineering에 너무 시간을 들이지 마세요!)
- Ordinal Encoding을 적용해 보세요.
- **(Urclass Quiz) 다음 특성들 중에 순서를 고려한 Ordinal Encoding이 필요해 보이는 특성을 고르세요.**
    1. opinion_h1n1_vacc_effective
    2. state
    3. marital
    4. employment_occupation


In [2]:
import pandas as pd
from google.colab import files

file = files.upload()

Saving submission.csv to submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv
Saving train_labels.csv to train_labels.csv


In [93]:
submission = pd.read_csv('submission.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
train_labels = pd.read_csv('train_labels.csv')

In [94]:
from sklearn.model_selection import train_test_split

target = 'vacc_h1n1_f'
train = pd.merge(train, train_labels[target], left_index=True, right_index=True)

train, val = train_test_split(train, train_size=0.80, test_size=0.20, stratify=train[target], random_state=2)

In [95]:
def feature_engineering(df):
    behaviorals = [col for col in df.columns if 'behavior' in col]
    df['behavior'] = df[behaviorals].sum(axis = 1)
    df.drop(columns = behaviorals, inplace = True)

    dels = [col for col in df.columns if 'seas' in col or 'employment' in col]
    df.drop(columns = dels, inplace = True)

    return df

train = feature_engineering(train)
val = feature_engineering(val)
test = feature_engineering(test)

X_train = train.drop(target, axis=1)
y_train = train[target]
X_val = val.drop(target, axis=1)
y_val = val[target]

In [100]:
from category_encoders import OrdinalEncoder, OneHotEncoder

columns = [col for col in X_train.columns if 'opinion' in col]
encoder = OrdinalEncoder()
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(test)
X_val = encoder.transform(X_val)

### 2) 랜덤포레스트 모델을 적용한 후의 결과를 캐글에 제출하세요.
- 랜덤포레스트를 적용하고 성능이 오히려 떨어졌을 수도 있습니다! 만약 그렇다면 이유를 본인 논리로 분석해 보세요.
- **(Urclass Quiz) 캐글 Leaderboard의 본인 Score를 제출하세요.**

In [101]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

pipe = make_pipeline(
    OneHotEncoder(),
    StandardScaler(),
    SimpleImputer(),
    RandomForestClassifier(random_state=10, n_jobs=-1, oob_score=True)
)

pipe.fit(X_train, y_train)
pipe.score(X_val, y_val)
X_train

,h1n1_concern,h1n1_knowledge,doctor_recc_h1n1,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,...,marital,rent_own_r,census_region,census_msa,n_adult_r,household_children,n_people_r,hhs_region,state,behavior
20365,1.0,1.0,0.0,1.0,0.0,NaN,0.0,1,1,1,...,2.0,1.0,1,1,4.0,0.0,4.0,2,1,0.0
29138,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1,2,2,...,2.0,2.0,2,2,2.0,2.0,4.0,7,2,5.0
36128,2.0,2.0,0.0,1.0,0.0,1.0,0.0,2,3,3,...,2.0,1.0,4,1,1.0,0.0,1.0,10,3,4.0
39793,2.0,1.0,0.0,0.0,0.0,1.0,0.0,3,1,2,...,1.0,1.0,2,3,2.0,0.0,2.0,7,2,3.0
32973,0.0,1.0,0.0,0.0,0.0,NaN,0.0,4,1,1,...,1.0,1.0,2,2,2.0,3.0,5.0,7,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8747,NaN,NaN,0.0,0.0,NaN,1.0,NaN,6,4,6,...,NaN,1.0,2,1,4.0,1.0,5.0,5,22,0.0
4412,3.0,2.0,1.0,0.0,0.0,NaN,0.0,2,2,5,...,1.0,1.0,2,1,2.0,0.0,2.0,5,30,6.0
19064,NaN,NaN,0.0,0.0,NaN,NaN,NaN,6,4,6,...,NaN,1.0,3,2,2.0,1.0,3.0,4,5,0.0
40906,NaN,NaN,0.0,0.0,NaN,1.0,NaN,6,4,6,...,NaN,1.0,4,2,3.0,1.0,4.0,8,21,0.0


In [102]:
X_test

,h1n1_concern,h1n1_knowledge,doctor_recc_h1n1,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,...,marital,rent_own_r,census_region,census_msa,n_adult_r,household_children,n_people_r,hhs_region,state,behavior
0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,Not At All Effective,Very Low,Not At All Worried,...,2.0,1.0,2,1,2.0,0.0,2.0,7,29,0.0
1,3.0,0.0,0.0,NaN,NaN,NaN,NaN,Dont Know,Dont Know,Dont Know,...,NaN,NaN,3,3,1.0,0.0,1.0,6,17,3.0
2,1.0,1.0,0.0,0.0,0.0,NaN,0.0,Very Effective,Very Low,Not Very Worried,...,2.0,1.0,1,2,1.0,0.0,1.0,1,41,3.0
3,2.0,2.0,0.0,0.0,0.0,NaN,0.0,Very Effective,Somewhat High,Not Very Worried,...,1.0,1.0,4,1,2.0,0.0,2.0,9,35,0.0
4,2.0,1.0,0.0,0.0,0.0,NaN,0.0,Somewhat Effective,Somewhat Low,Not Very Worried,...,2.0,1.0,1,2,1.0,0.0,1.0,1,41,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28099,0.0,1.0,0.0,1.0,0.0,1.0,0.0,Somewhat Effective,Somewhat Low,Not Very Worried,...,2.0,2.0,2,3,3.0,1.0,4.0,7,36,6.0
28100,3.0,2.0,0.0,1.0,0.0,1.0,0.0,Somewhat Effective,Somewhat Low,Somewhat Worried,...,1.0,1.0,3,1,2.0,0.0,2.0,4,18,4.0
28101,1.0,1.0,0.0,0.0,0.0,NaN,0.0,Very Effective,Very Low,Not Very Worried,...,1.0,1.0,4,1,3.0,0.0,3.0,9,35,3.0
28102,2.0,1.0,0.0,1.0,0.0,1.0,0.0,Somewhat Effective,Very High,Not At All Worried,...,1.0,99.0,2,3,2.0,2.0,4.0,7,50,3.0


In [103]:
X_val

,h1n1_concern,h1n1_knowledge,doctor_recc_h1n1,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,...,marital,rent_own_r,census_region,census_msa,n_adult_r,household_children,n_people_r,hhs_region,state,behavior
18885,NaN,NaN,0.0,1.0,NaN,1.0,NaN,6,4,6,...,NaN,1.0,3,1,2.0,2.0,4.0,6,46,0.0
8587,1.0,2.0,0.0,0.0,0.0,NaN,1.0,5,1,1,...,1.0,1.0,2,2,2.0,0.0,2.0,7,50,3.0
17917,NaN,NaN,0.0,1.0,NaN,0.0,NaN,6,4,6,...,NaN,1.0,4,1,3.0,1.0,4.0,9,9,0.0
27985,3.0,1.0,1.0,1.0,0.0,1.0,0.0,3,2,4,...,2.0,2.0,3,1,2.0,0.0,2.0,4,13,6.0
917,2.0,0.0,0.0,1.0,0.0,NaN,0.0,5,3,1,...,1.0,1.0,2,1,2.0,0.0,2.0,5,45,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25383,3.0,1.0,1.0,1.0,0.0,1.0,0.0,3,2,5,...,1.0,1.0,3,1,2.0,0.0,2.0,3,14,5.0
37833,2.0,1.0,0.0,0.0,0.0,1.0,0.0,3,3,2,...,NaN,1.0,4,3,2.0,0.0,2.0,8,25,2.0
6921,2.0,2.0,0.0,0.0,0.0,NaN,0.0,3,1,1,...,2.0,2.0,3,1,2.0,0.0,2.0,4,11,3.0
2706,1.0,1.0,0.0,0.0,1.0,1.0,0.0,2,3,2,...,1.0,1.0,4,3,2.0,0.0,2.0,9,9,3.0


In [89]:
pred = pipe.predict(test)

ValueError: ignored

## 🔥 도전과제(Github - Discussion)

### 3) 수업에 사용하지 않은 다른 종류의 [category_encoders](http://contrib.scikit-learn.org/category_encoders/)을 2개 이상 사용해 결과를 공유해 보시고, 다음 질문에 대해 서로 논의해 보세요. 

- 사용하신 encoder는 각각 어떤 장단점을 갖고 있으며, 어떤 상황에서 사용하면 좋을까요? 

### 4) 왜 트리모델에서는 ordinal encoding을 주로 사용하며 (one-hot encoding대신), 범주형 자료를 ordinal encoding으로 사용해도 되는 이유는 무엇이라고 생각하시는지 논의해 보세요


## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)